In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from IPython.display import Image

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename


from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np
import random

# Uses ML to find the symbol from the image put in
def human():
  # Disable scientific notation for clarity
  np.set_printoptions(suppress=True)

  # Load the model
  model = load_model("keras_model.h5", compile=False)

  # Load the labels
  class_names = open("labels.txt", "r").readlines()

  # Create the array of the right shape to feed into the keras model
  # The 'length' or number of images you can put into the array is
  # determined by the first position in the shape tuple, in this case 1
  data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

  # Replace this with the path to your image
  camera_photo = take_photo()
  image = Image.open(camera_photo).convert("RGB")
  # image = Image.open("scissors2.jpg").convert("RGB")


  # resizing the image to be at least 224x224 and then cropping from the center
  size = (224, 224)
  image = ImageOps.fit(image, size, Image.LANCZOS)

  # turn the image into a numpy array
  image_array = np.asarray(image)

  # Normalize the image
  normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

  # Load the image into the array
  data[0] = normalized_image_array

  # Predicts the model
  prediction = model.predict(data)
  index = np.argmax(prediction)
  class_name = class_names[index]
  confidence_score = prediction[0][index]

  # Print prediction and confidence score
  # print("Class:", class_name[2:], end="")
  print("ML Confidence Score:", confidence_score)
  return index

# generates a random play for the computer (pc)
def computer():
  comp = random.randint(0,2)
  return comp

# casting number guess to string for better understanding
def getWording(num):
  wording = ""
  if num == 0:
      wording = "Rock"
  elif num == 1:
      wording = "Paper"
  elif num == 2:
      wording = "Scissors"
  return wording

# does the logic of the two choices
def main():
  pc = computer()
  man = human()

  pcWording = getWording(pc)
  manWording = getWording(man)

  print("PC Play: " + pcWording)
  print("Your play (ML): " + manWording)

  if pc == man:
    print("DRAW")

  elif pc == 0 and man == 1:
    print("You win, paper eats rock")

  elif pc == 0 and man == 2:
    print("You lose, rock beats scissors")

  elif pc == 1 and man == 2:
    print("You win, scissors cut paper")

  elif pc == 1 and man == 0:
    print("You lose, paper eats rock")

  elif pc == 2 and man == 0:
    print("You win, rock smashes scissors")

  elif pc == 2 and man == 1:
    print("You lose, scissors cut paper")

# calls the main function to execute the creation
main()

try:
  filename = take_photo()
  print('Saved to {}'.format(filename))

  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

<IPython.core.display.Javascript object>

1/1 [==============================] - 1s 835ms/step
ML Confidence Score: 1.0
PC Play: Paper
Your play (ML): Scissors
You win, scissors cut paper


<IPython.core.display.Javascript object>

KeyboardInterrupt: ignored